In [ ]:
#Q1
import requests
from bs4 import BeautifulSoup
import pandas as pd

all_titles = []
all_prices = []
all_availability = []
all_ratings = []

page = 1
base_url = "https://books.toscrape.com/catalogue/page-{}.html"

while True:
    url = base_url.format(page)
    response = requests.get(url)
    if response.status_code != 200:
        break
    soup = BeautifulSoup(response.text, "html.parser")
    books = soup.find_all("article", class_="product_pod")
    if not books:
        break
    for book in books:
        title = book.h3.a.attrs['title']
        price = book.find("p", class_="price_color").text
        availability = book.find("p", class_="instock availability").text.strip()
        rating = book.p.attrs['class'][1]
        all_titles.append(title)
        all_prices.append(price)
        all_availability.append(availability)
        all_ratings.append(rating)
    page += 1

df_books = pd.DataFrame({
    "Title": all_titles,
    "Price": all_prices,
    "Availability": all_availability,
    "StarRating": all_ratings,
})

df_books.to_csv("books.csv", index=False)
print(df_books.head())


                                   Title    Price Availability StarRating
0                   A Light in the Attic  Â£51.77     In stock      Three
1                     Tipping the Velvet  Â£53.74     In stock        One
2                             Soumission  Â£50.10     In stock        One
3                          Sharp Objects  Â£47.82     In stock       Four
4  Sapiens: A Brief History of Humankind  Â£54.23     In stock       Five


In [ ]:
#Q2
!pip install selenium
!apt-get update # for driver installation
!apt install chromium-chromedriver


from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import time

options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(options=options)

driver.get("https://www.imdb.com/chart/top")
time.sleep(3)

ranks = []
titles = []
years = []
ratings = []

rows = driver.find_elements(By.CSS_SELECTOR, "tbody.lister-list tr")
for rank, row in enumerate(rows, start=1):
    title_col = row.find_element(By.CLASS_NAME, "titleColumn")
    title = title_col.find_element(By.TAG_NAME, "a").text
    year = title_col.find_element(By.CLASS_NAME, "secondaryInfo").text.strip("()")
    rating = row.find_element(By.CLASS_NAME, "imdbRating").text.strip()
    ranks.append(rank)
    titles.append(title)
    years.append(year)
    ratings.append(rating)

driver.quit()

df_movies = pd.DataFrame({
    "Rank": ranks,
    "Title": titles,
    "Year": years,
    "IMDB_Rating": ratings,
})

df_movies.to_csv("imdbtop250.csv", index=False)
print(df_movies.head())



Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:2 http://security.ubuntu.com/ubuntu jammy-security InRelease
Get:3 https://cli.github.com/packages stable InRelease [3,917 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:8 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 3,917 B in 1s (3,253 B/s)
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (

In [ ]:
#Q3
import requests
from bs4 import BeautifulSoup
import pandas as pd

url = "https://www.timeanddate.com/weather/"
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")

cities = []
temps = []
conditions = []

tables = soup.find_all('table')
found = False
for table in tables:
    if table.tbody:
        rows = table.tbody.find_all('tr')
        for row in rows:
            columns = row.find_all('td')
            if len(columns) >= 4:
                city = columns[0].get_text(strip=True)
                temp = columns[1].get_text(strip=True)
                cond = columns[2].get_text(strip=True)
                if city and temp and cond:
                    cities.append(city)
                    temps.append(temp)
                    conditions.append(cond)
                    found = True
if not found:
    print("No weather data table found. HTML structure might have changed.")

df_weather = pd.DataFrame({
    "City": cities,
    "Temperature": temps,
    "Condition": conditions,
})

df_weather.to_csv("weather.csv", index=False)
print(df_weather.head())


No weather data table found. HTML structure might have changed.
Empty DataFrame
Columns: [City, Temperature, Condition]
Index: []
